### PROJECT-4. Решение комплексной бизнес-задачи


##### ЦЕЛЬ ПРОЕКТА: 
Подготовить основу рекомендательной системы курсов для онлайн-школы MasterMind.

##### ЗАДАЧИ:
1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями.

1. Создаем SQL-запрос для получения необходимых данных:

In [ ]:
WITH count_users AS (
  SELECT 
    user_id, 
    COUNT (resource_id) AS count_cours 
  FROM 
    final.carts AS c 
    JOIN final.cart_items AS ci ON ci.cart_id = c.id 
  WHERE 
    state = 'successful' 
    AND resource_type = 'Course' 
  GROUP BY 
    user_id 
  HAVING 
    COUNT (DISTINCT resource_id) > 1
) 
SELECT 
  DISTINCT count_users.user_id, 
  resource_id 
FROM 
  final.carts AS c 
  JOIN final.cart_items AS ci ON ci.cart_id = c.id 
  RIGHT JOIN count_users ON c.user_id = count_users.user_id 
WHERE 
  state = 'successful' 
  AND resource_type = 'Course' 
ORDER BY 
  count_users.user_id, 
  resource_id

2. Обрабатываем данные средствами python:

In [32]:
#Загружаем библиотеки и данные, которые мы получили ранее с помощью sql-запроса
import pandas as pd
import numpy as np
import itertools as it
import collections as cl
sales_df = pd.read_csv('users.csv')

In [36]:
#Группируем покупки курсов по пользователям
courses_df = sales_df.groupby(['user_id'])['resource_id'].apply(lambda x:list(np.unique(x))).reset_index()
courses_df

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


In [33]:
#Разбиваем все покупки курсов по парам
courses = list(courses_df['resource_id'].apply(lambda y: list(it.combinations(y, 2))))
from itertools import chain
courses_list = list(chain.from_iterable(courses))
print(courses_list)

[(516, 1099), (356, 357), (356, 1125), (357, 1125), (553, 1147), (361, 1138), (356, 357), (1125, 1140), (551, 745), (553, 745), (551, 1138), (553, 568), (514, 517), (514, 566), (517, 566), (363, 511), (363, 562), (363, 563), (511, 562), (511, 563), (562, 563), (568, 745), (509, 553), (509, 745), (553, 745), (1125, 1144), (509, 568), (509, 672), (568, 672), (516, 552), (356, 552), (357, 571), (568, 745), (509, 516), (509, 568), (516, 568), (513, 1141), (571, 1125), (551, 552), (551, 744), (551, 862), (551, 1138), (552, 744), (552, 862), (552, 1138), (744, 862), (744, 1138), (862, 1138), (361, 1138), (356, 679), (571, 745), (571, 1099), (745, 1099), (509, 516), (509, 568), (509, 1099), (516, 568), (516, 1099), (568, 1099), (517, 750), (800, 1125), (569, 840), (509, 568), (745, 1125), (509, 514), (509, 551), (509, 745), (514, 551), (514, 745), (551, 745), (571, 765), (745, 1187), (363, 566), (513, 1125), (1100, 1103), (502, 564), (502, 865), (502, 1103), (564, 865), (564, 1103), (865, 110

In [40]:
#Считаем, сколько раз встречается каждая пара
from collections import Counter
courses_pairs = Counter(courses_list)
courses_dict = dict(courses_pairs)
print(courses_dict)

{(516, 1099): 25, (356, 357): 100, (356, 1125): 44, (357, 1125): 52, (553, 1147): 16, (361, 1138): 40, (1125, 1140): 1, (551, 745): 138, (553, 745): 212, (551, 1138): 14, (553, 568): 83, (514, 517): 10, (514, 566): 138, (517, 566): 21, (363, 511): 99, (363, 562): 77, (363, 563): 33, (511, 562): 55, (511, 563): 19, (562, 563): 53, (568, 745): 102, (509, 553): 48, (509, 745): 59, (1125, 1144): 22, (509, 568): 46, (509, 672): 5, (568, 672): 4, (516, 552): 12, (356, 552): 7, (357, 571): 112, (509, 516): 35, (516, 568): 54, (513, 1141): 34, (571, 1125): 122, (551, 552): 177, (551, 744): 16, (551, 862): 8, (552, 744): 8, (552, 862): 6, (552, 1138): 4, (744, 862): 2, (744, 1138): 1, (862, 1138): 3, (356, 679): 8, (571, 745): 22, (571, 1099): 2, (745, 1099): 53, (509, 1099): 31, (568, 1099): 53, (517, 750): 34, (800, 1125): 4, (569, 840): 204, (745, 1125): 15, (509, 514): 8, (509, 551): 15, (514, 551): 200, (514, 745): 38, (571, 765): 83, (745, 1187): 32, (363, 566): 20, (513, 1125): 1, (1100,

In [ ]:
#Создаем функцию рекомендаций, которая на вход получает id курса и возвращает два самых часто продаваемых курса с тем, что получен на вход
def recomendations(course_id):
    recomm_list = []
    for i in courses_dict.keys():
        if i[0] == course_id:
            recomm_list.append((i, courses_dict[i]))
        elif i[1] == course_id:
            recomm_list.append((i, courses_dict[i]))
    recomm_list = sorted(recomm_list, key = lambda a: a[1], reverse=True)
    return recomm_list[:2]

In [41]:
#Создаем список с уникальными id курсов
сourse_list = np.unique(sales_df['resource_id']).tolist()

In [48]:
#Определяем самую популярную пару курсов
courses_pairs.most_common(1)

[((551, 566), 797)]

3. Создаем итоговую таблицу рекомендаций:

На основании подсчета количества встречаемости пар курсов, выбрали минимальную границу - 10


Для остальных курсов будем рекомендовать самые популярные курсы (551, 566)

In [49]:
rec_list = []
recommendations_df = pd.DataFrame(rec_list, columns=['first_rec', 'second_rec'])

min_value = 10

for course in сourse_list:
    if recomendations(course)[0][1]>= min_value:
        rec_1 = (set(recomendations(course)[0][0]) - set([course])).pop()
        if recomendations(course)[1][1] >= min_value:
            rec_2 = (set(recomendations(course)[1][0])-set([course])).pop()
        else:
            rec_2 = 551
        recommendations_df.loc[course] = [rec_1, rec_2]
    elif recomendations(course)[0][1] < min_value:
        recommendations_df.loc[course] = [551, 566]
        
recommendations_df.head(10)

,first_rec,second_rec
356,571,357
357,571,356
358,570,752
359,570,358
360,745,516
361,551,1138
362,363,551
363,511,562
364,551,566
365,551,566
